# Linear Regression & Gradient Descent

In this notebook we will work with the diabetes dataset 
https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html
https://web.stanford.edu/~hastie/Papers/LARS/diabetes.data (raw data, not standardised)

The data set contains 10 baseline variables (X1, X2, X3....X10) for 442 patients with diabetes: age, sex, body mass index, average blood pressure, and six blood serum measurements. The target variable (y) indicates the progression of the dieases one year after baseline.

Luckily, this database is available in sklearn, so we need only to load it. 
Even more luckily, the sklearn database is already standardised (compare with the raw data above)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

In [2]:
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

We will consider a subset of the dataset, specifically only the variables: body mass index, average blood pressure and one blood serum measurement. Feel free to include more dimensions in your model. 

In [3]:
diabetes_X = diabetes_X[:,2:5] #select only the 3 relevant features we want to consider

First thing, we split our dataset in training set and test set (z.B 80 data points are left as a test).
We will fit(train) our model on the training dataset and then we will evaluate how they really perform on test data.
The performance on the test data is fundamental to see how models generalise.

In [4]:
diabetes_X_train = diabetes_X[:-80]
diabetes_X_test = diabetes_X[-80:]

diabetes_y_train = diabetes_y[:-80]
diabetes_y_test = diabetes_y[-80:]

In this Notebook we want to first compare several methods to determine the best parameters for our model. 
Let's start with the LinearRegression() function from sklearn.
The LinearRegression class is based on the scipy.linalg.lstsq() function (the name stands for "least squares"), this compute the optimal paramters'values using the least square exact method.

In [5]:
regr = linear_model.LinearRegression()


regr.fit(diabetes_X_train, diabetes_y_train)

# The model's parameters
print('Parameters (theta_1, theta_2, theta_3, theta_0): \n', regr.coef_, regr.intercept_)

Parameters (theta_1, theta_2, theta_3, theta_0): 
 [805.69232777 363.70203872  52.14450364] 152.53452637832743


### Batch Gradient Descent

Let's now try to compute the same parameters using the Batch Gradient Descent. 

In [6]:
eta = 0.1  # learning rate
n_iterations = 100000 #try with 1000, 10000, 100000...
N = len(diabetes_X)

theta_0 = np.random.randn(1) # random initialization
theta_1 = np.random.randn(1)
theta_2 = np.random.randn(1)
theta_3 = np.random.randn(1)

X1= diabetes_X_train[:,0]
X2= diabetes_X_train[:,1]
X3= diabetes_X_train[:,2]

Code here the formulas for the batch gradient descent

In [7]:
for iteration in range(n_iterations):

    
    gradient_theta_3 = 1/N * np.sum((theta_0 + theta_1*X1 + theta_2*X2 + theta_3*X3  - diabetes_y_train)*X3)
    gradient_theta_2 = 1/N * np.sum((theta_0 + theta_1*X1 + theta_2*X2 + theta_3*X3  - diabetes_y_train)*X2)
    gradient_theta_1 = 1/N * np.sum((theta_0 + theta_1*X1 + theta_2*X2 + theta_3*X3  - diabetes_y_train)*X1)
    gradient_theta_0 = 1/N * np.sum((theta_0 + theta_1*X1 + theta_2*X2 + theta_3*X3  - diabetes_y_train))
    
    
    theta_3 = theta_3 - eta * gradient_theta_3
    theta_2 = theta_2 - eta * gradient_theta_2                                
    theta_1 = theta_1 - eta * gradient_theta_1
    theta_0 = theta_0 - eta * gradient_theta_0
    
    

In [8]:
#Let's see hwo the parameters look like 
print(theta_1,theta_2,theta_3, theta_0)

[805.68576868] [363.70743351] [52.14566178] [152.53451874]


Compare with the values obtained with the function LinearRegression().
Try to change the number of iterations and/or eta to get closer to the exact values.

### Stochastic Gradient Descent
We try now to see how the results change if we use the SGD approach. We use here the sklearn function SGDRegressor.
Check out here the syntax
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html
General Infos:
https://scikit-learn.org/stable/modules/sgd.html#regression

In [9]:
from sklearn.linear_model import SGDRegressor

In [10]:
sgd_reg = SGDRegressor(max_iter=1000000, eta0=0.01)
sgd_reg.fit(diabetes_X_train, diabetes_y_train)

SGDRegressor(max_iter=1000000)

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html

In [11]:
sgd_reg.coef_, sgd_reg.intercept_

(array([627.88548258, 368.84550492, 123.33762085]), array([152.37568479]))

How does this compare with the previous solutions? Try to adapt also here eta and number of iterations. 
How does it compare in terms of velocity with the Batch Gradient Descent?

Try to run the previous 2 cells again. Do you get always the same result?

### Using a linear regression model

After you found the best parameters for your linear model, you can use them on the test data to predict values and evaluate how well your model perform, that is how much your predictions are off from the true y values of your test data.
Let's start with the model obtained with LinearRegression() and then do the same with the linear regression obtained via SGD. 
How do the 2 compare on the test data?

In [12]:
from sklearn.metrics import mean_squared_error

In [13]:
diabetes_y_pred_EasyLinReg = regr.predict(diabetes_X_test)
diabetes_y_pred_SGD = sgd_reg.predict(diabetes_X_test)

In [14]:
print('LinearRegression() Mean squared error: %.2f' % mean_squared_error(diabetes_y_test, diabetes_y_pred_EasyLinReg))
print('SGD() Mean squared error: %.2f' % mean_squared_error(diabetes_y_test, diabetes_y_pred_SGD))

LinearRegression() Mean squared error: 3327.59
SGD() Mean squared error: 3357.92


In [15]:
#Source: Notebook adapted from A.Geron, Hands On ML with Scikit-Learn, Keras und Tensorflow, O'Reilly